# **Taller Segmentación de Formas**

In [5]:
!pip install opencv-python matplotlib imageio

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio

### *Crear carpeta de salida*

In [7]:
os.makedirs("resultados", exist_ok=True)

### *Cargar imagen*

In [8]:
img = cv2.imread("flor.jpg")
if img is None:
    raise FileNotFoundError("No se encontró flor.jpg en el directorio.")

## **1. Convertir a escala de grises**

In [9]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite("resultados/01_gris.jpg", gray)

True

## **2. Binarización fija**

In [10]:
_, thresh_fijo = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
cv2.imwrite("resultados/02_binarizacion_fija.jpg", thresh_fijo)

True

## **3. Binarización adaptativa**

In [11]:
thresh_adapt = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                     cv2.THRESH_BINARY_INV, 11, 5)
cv2.imwrite("resultados/03_binarizacion_adaptativa.jpg", thresh_adapt)

True

## **4. Detectar contornos**

In [12]:
contornos, _ = cv2.findContours(thresh_adapt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contornos
img_contornos = img.copy()

areas = []
perimetros = []
for cnt in contornos:
    area = cv2.contourArea(cnt)
    perim = cv2.arcLength(cnt, True)
    if area < 100: continue  # descartar ruido pequeño

    areas.append(area)
    perimetros.append(perim)

    # Dibujar contorno
    cv2.drawContours(img_contornos, [cnt], -1, (0, 255, 0), 2)

    # Bounding box
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img_contornos, (x, y), (x+w, y+h), (255, 0, 0), 1)

    # Centroide
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv2.circle(img_contornos, (cx, cy), 3, (0, 0, 255), -1)
        cv2.putText(img_contornos, f"({cx},{cy})", (cx + 5, cy),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)

cv2.imwrite("resultados/04_contornos_dibujados.jpg", img_contornos)

True

## **5. Calcular métricas**

In [13]:
num_formas = len(areas)
area_prom = np.mean(areas) if areas else 0
perim_prom = np.mean(perimetros) if perimetros else 0

print(f"Número de formas: {num_formas}")
print(f"Área promedio: {area_prom:.2f}")
print(f"Perímetro promedio: {perim_prom:.2f}")

Número de formas: 41
Área promedio: 545.04
Perímetro promedio: 356.60


## **6. Crear GIF**

In [ ]:
gif_imgs = [
    "resultados/01_gris.jpg",
    "resultados/02_binarizacion_fija.jpg",
    "resultados/03_binarizacion_adaptativa.jpg",
    "resultados/04_contornos_dibujados.jpg"
]

base_shape = cv2.imread(gif_imgs[0]).shape[:2]  # alto, ancho
frames = []
for path in gif_imgs:
    img = cv2.imread(path)
    img_resized = cv2.resize(img, (base_shape[1], base_shape[0]))
    frames.append(img_resized)
imageio.mimsave("resultados/segmentacion.gif", frames, duration=2000)